In [1]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [31]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

## Step 1a - Indexing (Document Ingestion)

In [8]:
# The data has been loaded from youtube transcript

video_id = "cMuif_hJGPI"

try:
    api = YouTubeTranscriptApi()
    transcript = api.fetch(video_id, languagecript.ipynbs=["en"])
    print("Transcript fetched successfully!")
except TranscriptsDisabled:
    print("❌ Transcript not available for this video.")
    transcript = None

Transcript fetched successfully!


In [9]:
transcript.snippets

[FetchedTranscriptSnippet(text='On the table right now are four of the', start=0.08, duration=3.12),
 FetchedTranscriptSnippet(text='same phone. The first is loaded with', start=1.68, duration=3.679),
 FetchedTranscriptSnippet(text='chat GBT. Second is Google Gemini. Third', start=3.2, duration=4.079),
 FetchedTranscriptSnippet(text='is Perplexity which prides itself on', start=5.359, duration=3.761),
 FetchedTranscriptSnippet(text='giving accurate and trusted answers to', start=7.279, duration=3.841),
 FetchedTranscriptSnippet(text='any question. And finally Grock, which', start=9.12, duration=4.479),
 FetchedTranscriptSnippet(text='is trained on data from X. And so my', start=11.12, duration=3.92),
 FetchedTranscriptSnippet(text="guess is that it's going to be a lot", start=13.599, duration=3.68),
 FetchedTranscriptSnippet(text='more unfiltered. These are for the', start=15.04, duration=4.48),
 FetchedTranscriptSnippet(text='average consumer the four best AI chat', start=17.279, dura

In [10]:
len(transcript.snippets)

767

In [11]:
li = []
for snippet in transcript.snippets:
    li.append(snippet.text)
transcript = ' '.join(li)
transcript

'On the table right now are four of the same phone. The first is loaded with chat GBT. Second is Google Gemini. Third is Perplexity which prides itself on giving accurate and trusted answers to any question. And finally Grock, which is trained on data from X. And so my guess is that it\'s going to be a lot more unfiltered. These are for the average consumer the four best AI chat bots you can get. But you\'re only going to need one of them. So which one is the most accurate? Which one is the fastest? Which one should you be paying for to make your life easier? Let\'s kick things off with some problem solving. So, I drive a Honda Civic 2017. How many of the Aerolite 29in hard shell and these are all the dimensions suitcases would I be able to fit in the boot? Oh my goodness. Each one is literally given paragraphs and paragraphs of reasoning, especially Grog. What on earth is this? By the way, we have actually tested this ourselves in person and the correct answer is two if you want to ac

## Step 1b - Indexing (Text Splitting)

In [12]:
# the large text splitted into smaller chunks

splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

chunks = splitter.create_documents([transcript])

In [13]:
len(chunks)

37

In [14]:
chunks[0]

Document(metadata={}, page_content="On the table right now are four of the same phone. The first is loaded with chat GBT. Second is Google Gemini. Third is Perplexity which prides itself on giving accurate and trusted answers to any question. And finally Grock, which is trained on data from X. And so my guess is that it's going to be a lot more unfiltered. These are for the average consumer the four best AI chat bots you can get. But you're only going to need one of them. So which one is the most accurate? Which one is the fastest? Which one should you be paying for to make your life easier? Let's kick things off with some problem solving. So, I drive a Honda Civic 2017. How many of the Aerolite 29in hard shell and these are all the dimensions suitcases would I be able to fit in the boot? Oh my goodness. Each one is literally given paragraphs and paragraphs of reasoning, especially Grog. What on earth is this? By the way, we have actually tested this ourselves in person and the correct

In [15]:
chunks[1]

Document(metadata={}, page_content="given paragraphs and paragraphs of reasoning, especially Grog. What on earth is this? By the way, we have actually tested this ourselves in person and the correct answer is two if you want to actually be able to close that boot door. So, with that in mind, I'd say both Chad GBT and Google Gemini have the right idea. They both say that you could theoretically fit three, but in practice, more likely two. Perplexity is just straight up wrong. It says three and maybe four if you arrange them efficiently. And then I would actually argue Grock has the best answer because this guy just says two with complete confidence. No messing around. I want to make a cake. This is what I have. And then let's attach a photo of well, four ingredients that I definitely should be using. And then one dehydrated porchini mushrooms that it definitely shouldn't. So, oh my god, this is interesting. Every AI thinks this jar of mushrooms is something different. ChachiT thinks it'

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [16]:
#Generate the embeddings for the chunks

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = FAISS.from_documents(chunks, embedding=embeddings)

/tmp/ipykernel_5868/3600191191.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/rathan-r-nayak/Rathan/Machine Learning/machine-leaning/langchain_models/langchain_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/rathan-r-nayak/Rathan/Machine Learning/machine-leaning/langchain_models/langchain_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarn

In [17]:
vector_store.index_to_docstore_id

{0: 'b64fb45b-299c-482c-8fc9-b0d7e36ccb51',
 1: '29002af8-3c9e-42e9-b214-7f2cb6f7e83b',
 2: '4e32c27e-a3cf-4704-872d-4a272bf4f509',
 3: 'da559b5d-e12e-46c2-9ebe-e45d20935de1',
 4: '85cd4d74-bc05-4ec5-8b5a-6e516ec0c99a',
 5: '75f5a797-9a9d-4777-a372-c6b4384bb691',
 6: '1b3ea682-7319-4217-8645-327405a4d667',
 7: 'f6d323d8-765f-4409-894a-7c7d1e5c67d1',
 8: 'f3977fdf-5272-4c51-9f9f-a7c64e88aa34',
 9: '57359226-5d56-4c45-960f-075019c5f791',
 10: '73e88f6d-a5d8-41bf-a225-b40fdfc47392',
 11: 'df3117ab-9569-4abc-97ea-1935181ed3a1',
 12: '32caf2f7-1fd8-43dc-9083-b946d72fe458',
 13: 'c27e2160-d5fa-4172-bb87-1f96f746df97',
 14: '1ac8b344-6d39-40ce-85df-c1e3047fcd43',
 15: '952b0117-3104-460f-b567-848e40136856',
 16: 'd4f1d094-135c-48ef-9f49-4b276915a49a',
 17: '190af022-7859-44bd-9cc2-d3a80657db7a',
 18: '5eba0462-e63e-4b79-b287-5626db87e759',
 19: 'e63d905c-f065-45b3-9393-df844c62ef7a',
 20: 'fcb66676-f787-4edb-b4ed-dc4b0d5a2d5e',
 21: 'e3c25d94-b9e4-4b26-a3b5-7bf3fd3ba11b',
 22: '39613fb8-dd63-

In [18]:
vector_store.get_by_ids(['cc39003e-f3d3-4653-a0b8-298c9e905008'])

[]

## Step 2 - Retrieval

In [19]:
# User retrival to retrive the document

retriver = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':3})

In [20]:
retriver.invoke("Which apple series has best battery")

/home/rathan-r-nayak/Rathan/Machine Learning/machine-leaning/langchain_models/langchain_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(id='df3117ab-9569-4abc-97ea-1935181ed3a1', metadata={}, page_content="and extract all the information from that web page. Not to mention that Google isn't self-aware that it can't do this. It thinks it's looking at the M10 earphones, which I've never heard of the M10 earphones, but they definitely aren't the link. And then Plexity thinks the exact same link is the F9 earphones, which they also aren't. And then finally to see how up to date these are on what's happening in the moment. What's the highest power output charger that UG Green sells? Yes. Okay. Good. So this is at least working for a long time. The answer was 300 W. And only yesterday they announced a 500 W charger. So somewhat relieved to see that each AI has picked up on that because this news-based reporting was a distinct disadvantage of last generation AI. So, we've now seen how well each of these can put together existing information from the web. But if we want to take it a step further, the way to do that is

## Augmentation

In [21]:
llm = ChatGroq(model="llama-3.1-8b-instant")

prompt = PromptTemplate(template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables=["context","question"]
)

In [22]:
question = "Which AI is performed better"

docs = retriver.invoke(question)

/home/rathan-r-nayak/Rathan/Machine Learning/machine-leaning/langchain_models/langchain_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [23]:
docs

[Document(id='b64fb45b-299c-482c-8fc9-b0d7e36ccb51', metadata={}, page_content="On the table right now are four of the same phone. The first is loaded with chat GBT. Second is Google Gemini. Third is Perplexity which prides itself on giving accurate and trusted answers to any question. And finally Grock, which is trained on data from X. And so my guess is that it's going to be a lot more unfiltered. These are for the average consumer the four best AI chat bots you can get. But you're only going to need one of them. So which one is the most accurate? Which one is the fastest? Which one should you be paying for to make your life easier? Let's kick things off with some problem solving. So, I drive a Honda Civic 2017. How many of the Aerolite 29in hard shell and these are all the dimensions suitcases would I be able to fit in the boot? Oh my goodness. Each one is literally given paragraphs and paragraphs of reasoning, especially Grog. What on earth is this? By the way, we have actually tes

In [24]:
context = "\n\n".join(doc.page_content for doc in docs)
context

"On the table right now are four of the same phone. The first is loaded with chat GBT. Second is Google Gemini. Third is Perplexity which prides itself on giving accurate and trusted answers to any question. And finally Grock, which is trained on data from X. And so my guess is that it's going to be a lot more unfiltered. These are for the average consumer the four best AI chat bots you can get. But you're only going to need one of them. So which one is the most accurate? Which one is the fastest? Which one should you be paying for to make your life easier? Let's kick things off with some problem solving. So, I drive a Honda Civic 2017. How many of the Aerolite 29in hard shell and these are all the dimensions suitcases would I be able to fit in the boot? Oh my goodness. Each one is literally given paragraphs and paragraphs of reasoning, especially Grog. What on earth is this? By the way, we have actually tested this ourselves in person and the correct answer is two if you want to\n\nve

In [25]:
final_prompt = prompt.invoke({"context": context, "question": question})
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      On the table right now are four of the same phone. The first is loaded with chat GBT. Second is Google Gemini. Third is Perplexity which prides itself on giving accurate and trusted answers to any question. And finally Grock, which is trained on data from X. And so my guess is that it's going to be a lot more unfiltered. These are for the average consumer the four best AI chat bots you can get. But you're only going to need one of them. So which one is the most accurate? Which one is the fastest? Which one should you be paying for to make your life easier? Let's kick things off with some problem solving. So, I drive a Honda Civic 2017. How many of the Aerolite 29in hard shell and these are all the dimensions suitcases would I be able to fit in the boot? Oh my goodness. Each one is literally given p

In [26]:
answer = llm.invoke(final_prompt)
answer.content

'Based on the provided context, it seems that you are comparing the performance of four AI chatbots: Chat GBT, Google Gemini, Perplexity, and Grock. However, it appears that the context is actually comparing Veo (a cheese-related product) to Google\'s Nintendo Switch 2 fact-checking capabilities, but there is also a mention of Grock\'s performance.\n\nTo answer your question, I\'ll assume you are asking about the performance of the AI chatbots. Based on the provided text, it seems that:\n\n- Chat GBT is considered the best choice for the average customer due to its price and performance.\n- Google Gemini is mentioned as having a "sheer quality of the output" but is also more limited in terms of usage due to the tokens.\n- Perplexity is mentioned as giving accurate and trusted answers, but its performance is not directly compared to the other AI chatbots.\n- Grock\'s performance is not directly compared to the other AI chatbots, but it is mentioned as having a different training data an

## Chains

In [27]:
def getContext(docs):
    context_data = "\n\n".join(doc.page_content for doc in docs)
    return context_data

In [34]:
parellel_chain = RunnableParallel({
    'question' : RunnablePassthrough(),
    'context' : retriver | RunnableLambda(getContext)
})

In [35]:
parellel_chain.invoke("which Ai is better")

/home/rathan-r-nayak/Rathan/Machine Learning/machine-leaning/langchain_models/langchain_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'question': 'which Ai is better',
 'context': "On the table right now are four of the same phone. The first is loaded with chat GBT. Second is Google Gemini. Third is Perplexity which prides itself on giving accurate and trusted answers to any question. And finally Grock, which is trained on data from X. And so my guess is that it's going to be a lot more unfiltered. These are for the average consumer the four best AI chat bots you can get. But you're only going to need one of them. So which one is the most accurate? Which one is the fastest? Which one should you be paying for to make your life easier? Let's kick things off with some problem solving. So, I drive a Honda Civic 2017. How many of the Aerolite 29in hard shell and these are all the dimensions suitcases would I be able to fit in the boot? Oh my goodness. Each one is literally given paragraphs and paragraphs of reasoning, especially Grog. What on earth is this? By the way, we have actually tested this ourselves in person and

In [36]:
parser = StrOutputParser()

chain = parellel_chain | prompt | llm | parser

In [37]:
chain.invoke("Which AI is best")

/home/rathan-r-nayak/Rathan/Machine Learning/machine-leaning/langchain_models/langchain_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


'Based on the provided context, I would recommend Chat GBT as the best AI chatbot for the average consumer. The video states that Chat GBT is the "best choice for an AI chatbot right now for the average customer" due to its balance of accuracy, speed, and price, with a $20 a month tier.'